In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Technical Assessement THD')
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Technical Assessement THD


In [ ]:
import pandas as pd
import numpy as np
import datetime

import plotly.express as px

In [ ]:
df=pd.read_csv('store_sales_data_processed.csv')

df['date']=pd.to_datetime(df['date'])
df=df.rename(columns={'CLUSTER':'cluster'})
df=df.sort_values(['store_nbr','date'],ascending=[True,True])
df.dtypes


date               datetime64[ns]
store_nbr                   int64
sales                     float64
promotion_count             int64
city                       object
state                      object
store_type                 object
cluster                     int64
holiday_type               object
locale                     object
locale_name                object
description                object
transferred                object
year                        int64
month                       int64
day_of_mnth                 int64
day_of_week                object
daynum_of_week              int64
mnth_name                  object
dtype: object

### Overview of strategy

- In order to optimize for time, we need to look at the clusters on their own

- We need to check if holidays have a positive impact

- We need to look if any stores opened later than the 1st date provided

- 

In [ ]:
nonzero_sales=df[['store_nbr','sales','date']].sort_values(['store_nbr','date'],ascending=[True,True])
nonzero_sales=nonzero_sales[nonzero_sales['sales']>0]
nonzero_sales_min_dt=nonzero_sales.groupby(['store_nbr'])['date'].min().reset_index().rename(columns={'date':'min_non_zero_date'})
nonzero_sales_min_dt['starting_dt']=df['date'].min()
nonzero_sales_min_dt['days_from_start']=(nonzero_sales_min_dt['min_non_zero_date']-nonzero_sales_min_dt['starting_dt'])/np.timedelta64(1, 'D')

nonzero_sales_min_dt

#we can see that some stores start showing positive values many days after the min start date.
## Thus we'll need to flag these stores as closed till the respective start dates for those with a value >1

,store_nbr,min_non_zero_date,starting_dt,days_from_start
0,1,2013-01-02,2013-01-01,1.0
1,2,2013-01-02,2013-01-01,1.0
2,3,2013-01-02,2013-01-01,1.0
3,4,2013-01-02,2013-01-01,1.0
4,5,2013-01-02,2013-01-01,1.0
5,6,2013-01-02,2013-01-01,1.0
6,7,2013-01-02,2013-01-01,1.0
7,8,2013-01-02,2013-01-01,1.0
8,9,2013-01-02,2013-01-01,1.0
9,10,2013-01-02,2013-01-01,1.0


In [ ]:
##Running same check for closures

nonzero_sales=df[['store_nbr','sales','date']].sort_values(['store_nbr','date'],ascending=[True,True])
nonzero_sales=nonzero_sales[nonzero_sales['sales']>0]
nonzero_sales_max_dt=nonzero_sales.groupby(['store_nbr'])['date'].max().reset_index().rename(columns={'date':'max_non_zero_date'})
nonzero_sales_max_dt['ending_dt']=df['date'].max()
nonzero_sales_max_dt['days_from_start']=(nonzero_sales_max_dt['ending_dt']-nonzero_sales_max_dt['max_non_zero_date'])/np.timedelta64(1, 'D')
nonzero_sales_max_dt

## No action needed as all stores active till the last date 

,store_nbr,max_non_zero_date,ending_dt,days_from_start
0,1,2017-08-15,2017-08-15,0.0
1,2,2017-08-15,2017-08-15,0.0
2,3,2017-08-15,2017-08-15,0.0
3,4,2017-08-15,2017-08-15,0.0
4,5,2017-08-15,2017-08-15,0.0
5,6,2017-08-15,2017-08-15,0.0
6,7,2017-08-15,2017-08-15,0.0
7,8,2017-08-15,2017-08-15,0.0
8,9,2017-08-15,2017-08-15,0.0
9,10,2017-08-15,2017-08-15,0.0


In [ ]:
##Treating for later opened stores
later_opened_stores=nonzero_sales_min_dt[nonzero_sales_min_dt['days_from_start']>1]


##joining to main dataset

df1=pd.merge(df,later_opened_stores,on=['store_nbr'],how='left')
df1['starting_dt']=df.date.min()
df1.loc[(df1['min_non_zero_date'].isnull()),'min_non_zero_date']=df1['starting_dt']
df1['active']='Y'
df1.loc[(df1['date']<df1['min_non_zero_date']),'active']='N'
df1.reset_index(drop=True,inplace=True)

In [ ]:

store_type_dummies=pd.get_dummies(df1.store_type,prefix='st',drop_first=True)
store_type_dummies
df1=pd.concat([df1,store_type_dummies],axis=1)
df1

,date,store_nbr,sales,promotion_count,city,state,store_type,cluster,holiday_type,locale,...,daynum_of_week,mnth_name,min_non_zero_date,starting_dt,days_from_start,active,st_B,st_C,st_D,st_E
0,2013-01-01,1,0.000000,0,Quito,Pichincha,D,13,Holiday,National,...,2,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,1,0
1,2013-01-02,1,7417.148000,0,Quito,Pichincha,D,13,NaN,NaN,...,3,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,1,0
2,2013-01-03,1,5873.244001,0,Quito,Pichincha,D,13,NaN,NaN,...,4,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,1,0
3,2013-01-04,1,5919.879001,0,Quito,Pichincha,D,13,NaN,NaN,...,5,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,1,0
4,2013-01-05,1,6318.785010,0,Quito,Pichincha,D,13,Work Day,National,...,6,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90931,2017-08-11,54,8513.834000,166,El Carmen,Manabi,C,3,Transfer,National,...,5,Aug,2013-01-01,2013-01-01,NaN,Y,0,1,0,0
90932,2017-08-12,54,9139.678002,138,El Carmen,Manabi,C,3,NaN,NaN,...,6,Aug,2013-01-01,2013-01-01,NaN,Y,0,1,0,0
90933,2017-08-13,54,14246.827996,173,El Carmen,Manabi,C,3,NaN,NaN,...,0,Aug,2013-01-01,2013-01-01,NaN,Y,0,1,0,0
90934,2017-08-14,54,11882.994000,126,El Carmen,Manabi,C,3,NaN,NaN,...,1,Aug,2013-01-01,2013-01-01,NaN,Y,0,1,0,0


In [ ]:
##getting count of stores by cluster
cluster_cnt=df1.groupby(['cluster'])['store_nbr'].nunique().reset_index().rename(columns={'store_nbr':'store_cnt'})
cluster_cnt

,cluster,store_cnt
0,1,3
1,2,2
2,3,7
3,4,3
4,5,1
5,6,6
6,7,2
7,8,3
8,9,2
9,10,6


In [ ]:
##Trying for 1 cluster





##Try to fit for cluster 6

cl6=df1[df1['cluster']==17]
cl6=cl6[cl6['active']=='Y']
cl6.reset_index(drop=True,inplace=True)

cl6

,date,store_nbr,sales,promotion_count,city,state,store_type,cluster,holiday_type,locale,...,daynum_of_week,mnth_name,min_non_zero_date,starting_dt,days_from_start,active,st_B,st_C,st_D,st_E
0,2013-01-01,51,0.000000,0,Guayaquil,Guayas,A,17,Holiday,National,...,2,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,0,0
1,2013-01-02,51,15134.490990,0,Guayaquil,Guayas,A,17,NaN,NaN,...,3,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,0,0
2,2013-01-03,51,11170.057015,0,Guayaquil,Guayas,A,17,NaN,NaN,...,4,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,0,0
3,2013-01-04,51,12452.326000,0,Guayaquil,Guayas,A,17,NaN,NaN,...,5,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,0,0
4,2013-01-05,51,15046.247000,0,Guayaquil,Guayas,A,17,Work Day,National,...,6,Jan,2013-01-01,2013-01-01,NaN,Y,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2017-08-11,51,26070.018000,312,Guayaquil,Guayas,A,17,Transfer,National,...,5,Aug,2013-01-01,2013-01-01,NaN,Y,0,0,0,0
1680,2017-08-12,51,19532.964000,139,Guayaquil,Guayas,A,17,NaN,NaN,...,6,Aug,2013-01-01,2013-01-01,NaN,Y,0,0,0,0
1681,2017-08-13,51,25713.284030,145,Guayaquil,Guayas,A,17,NaN,NaN,...,0,Aug,2013-01-01,2013-01-01,NaN,Y,0,0,0,0
1682,2017-08-14,51,18932.967000,132,Guayaquil,Guayas,A,17,NaN,NaN,...,1,Aug,2013-01-01,2013-01-01,NaN,Y,0,0,0,0


In [ ]:

timevar=cl6[['year','month','daynum_of_week']].drop_duplicates(keep='first').reset_index(drop=True)

time_yr=pd.get_dummies(timevar.year,prefix='yr',drop_first=True)

timevar=pd.concat([timevar,time_yr],axis=1)

timevar['sin_mnth']=np.sin(np.deg2rad(df.month))

timevar['cos_mnth']=np.cos(np.deg2rad(df.month))

timevar['sin_dow']=np.sin(np.deg2rad(df.daynum_of_week))

timevar['cos_dow']=np.cos(np.deg2rad(df.daynum_of_week))

cl6=pd.merge(cl6,timevar,on=['year','month','daynum_of_week'],how='inner')
cl6.reset_index(drop=True,inplace=True)



In [ ]:
cl6

,date,store_nbr,sales,promotion_count,city,state,store_type,cluster,holiday_type,locale,...,yr_2014,yr_2015,yr_2016,yr_2017,sin_mnth,cos_mnth,sin_dow,cos_dow,outliers,scores
0,2013-01-01,51,0.000000,0,Guayaquil,Guayas,A,17,Holiday,National,...,0,0,0,0,0.017452,0.999848,0.034899,0.999391,Y,-0.027615
1,2013-01-08,51,10525.460009,0,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,0,0.017452,0.999848,0.034899,0.999391,N,0.088390
2,2013-01-15,51,10648.973980,0,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,0,0.017452,0.999848,0.034899,0.999391,N,0.089019
3,2013-01-22,51,9573.298010,0,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,0,0.017452,0.999848,0.034899,0.999391,N,0.080488
4,2013-01-29,51,8741.145000,0,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,0,0.017452,0.999848,0.034899,0.999391,N,0.065544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2017-08-12,51,19532.964000,139,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,1,0.017452,0.999848,0.000000,1.000000,N,0.011266
1680,2017-08-06,51,29140.969000,134,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,1,0.017452,0.999848,0.017452,0.999848,N,0.005663
1681,2017-08-13,51,25713.284030,145,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,1,0.017452,0.999848,0.017452,0.999848,N,0.024792
1682,2017-08-07,51,21202.605000,131,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,1,0.017452,0.999848,0.034899,0.999391,N,0.050172


In [ ]:
model_data=cl6[['sales','promotion_count','month','store_nbr','sin_mnth','cos_mnth','sin_dow','cos_dow','st_B','st_C','st_D','st_E']]

from sklearn.ensemble import IsolationForest
if_model=IsolationForest(n_estimators=200,max_samples=0.9,max_features=12,contamination=0.05,random_state=42)
if_model.fit(model_data)

cl6['outliers']=pd.Series(if_model.predict(model_data)).apply(lambda x: 'Y' if (x==-1) else 'N' )
score=if_model.decision_function(model_data)
cl6['scores']=score
fig=px.scatter(cl6, x="date", y="sales", color='outliers', facet_row="store_nbr")

fig.show()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



In [ ]:
fig_hist = px.histogram(cl6, x="scores", nbins=50)
fig_hist.show()

In [ ]:
cl6.query('outliers=="Y"')

,date,store_nbr,sales,promotion_count,city,state,store_type,cluster,holiday_type,locale,...,yr_2014,yr_2015,yr_2016,yr_2017,sin_mnth,cos_mnth,sin_dow,cos_dow,outliers,scores
0,2013-01-01,51,0.000000,0,Guayaquil,Guayas,A,17,Holiday,National,...,0,0,0,0,0.017452,0.999848,0.034899,0.999391,Y,-0.024966
79,2013-03-12,51,6446.291000,0,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,0,0.017452,0.999848,0.104528,0.994522,Y,-0.017070
364,2014-01-01,51,0.000000,0,Guayaquil,Guayas,A,17,Holiday,National,...,1,0,0,0,0.052336,0.998630,0.034899,0.999391,Y,-0.033158
383,2014-01-05,51,28574.511990,0,Guayaquil,Guayas,A,17,NaN,NaN,...,1,0,0,0,0.052336,0.998630,0.104528,0.994522,Y,-0.011258
706,2014-12-23,51,36616.891000,40,Guayaquil,Guayas,A,17,Additional,National,...,1,0,0,0,0.104528,0.994522,0.052336,0.998630,Y,-0.016475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,2017-07-12,51,26194.840002,472,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,1,0.017452,0.999848,0.000000,1.000000,Y,-0.010347
1659,2017-07-19,51,26832.680000,437,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,1,0.017452,0.999848,0.000000,1.000000,Y,-0.003115
1675,2017-08-10,51,15364.387000,101,Guayaquil,Guayas,A,17,Holiday,National,...,0,0,0,1,0.017452,0.999848,0.087156,0.996195,Y,-0.001621
1676,2017-08-04,51,26565.046000,303,Guayaquil,Guayas,A,17,NaN,NaN,...,0,0,0,1,0.017452,0.999848,0.104528,0.994522,Y,-0.025065
